In [1]:
from sys import path
path.append("../")

# Exporting Landsat 8 to GeoTiff format  

> **Description**  
> The code in this notebook subsets a data cube, selects a specific set of variables, and then outputs that data into a GeoTIFF file. The goal is to be able to do external analyses of this data using other data analysis tools or GIS tools. The files would be reasonable in size, since we would restrict the region and parameters in the output.

----  

# Boiler Plate, Loading Data

> ### Import the Datacube

In [1]:
import datacube
dc = datacube.Datacube(app = 'my_app', config = '/home/localuser/.datacube.conf')

>### Browse the available Data Cubes on the storage platform    
> You might want to learn more about what data is stored and how it is stored.


In [2]:
list_of_products = dc.list_products()
netCDF_products = list_of_products[list_of_products['format'] == 'NetCDF']
netCDF_products

,name,description,lon,lat,product_type,format,instrument,time,platform,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,
61,alos2_jjfast_scansar_tile_colombia,ALOS2 PALSAR JJFAST tile in DN format processe...,None,None,tile,NetCDF,PALSAR,None,ALOS_2,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
50,alos2_palsar_colombia,ALOS2 PALSAR tile in DN format processed for t...,None,None,gamma0,NetCDF,PALSAR,None,ALOS_2,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
51,alos2_palsar_kenya,ALOS2 PALSAR tile in DN format processed for t...,None,None,gamma0,NetCDF,PALSAR,None,ALOS_2,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
52,alos2_palsar_vietnam,ALOS2 PALSAR tile in DN format processed for t...,None,None,gamma0,NetCDF,PALSAR,None,ALOS_2,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
53,alos_palsar_colombia,ALOS PALSAR tile in DN format processed for th...,None,None,gamma0,NetCDF,PALSAR,None,ALOS,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
54,alos_palsar_kenya,ALOS PALSAR tile in DN format processed for th...,None,None,gamma0,NetCDF,PALSAR,None,ALOS,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
55,alos_palsar_vietnam,ALOS PALSAR tile in DN format processed for th...,None,None,gamma0,NetCDF,PALSAR,None,ALOS,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
14,gpm_imerg_gis_daily_global,Global NetCDF GPM IMERG GIS data,None,None,daily,NetCDF,GPM,None,GPM,EPSG:4326,"[-0.1, 0.1]","[90, 180]","(latitude, longitude)"
15,gpm_imerg_gis_monthly_global,Global NetCDF GPM IMERG GIS data,None,None,monthly,NetCDF,GPM,None,GPM,EPSG:4326,"[-0.1, 0.1]","[90, 180]","(latitude, longitude)"


>### Pick a product  
>Use the platform names from the previous block to select a small Data Cube. The data_access_api utility will give you lat, lon, and time bounds of your Data Cube.   

In [7]:
import utils.data_cube_utilities.data_access_api as dc_api  
api = dc_api.DataAccessApi(config = '/home/localuser/.datacube.conf')

platform = "LANDSAT_8"
product = "ls8_lasrc_vietnam"

# Get Coordinates
coordinates = api.get_full_dataset_extent(platform = platform, product = product)
times = coordinates["time"].values 

latitude_extents = (min(coordinates['latitude'].values),max(coordinates['latitude'].values))
print( latitude_extents )
longitude_extents = (min(coordinates['longitude'].values),max(coordinates['longitude'].values))
print( longitude_extents )

(10.513927001104687, 12.611133863411238)
(106.79005909290998, 108.91906631627438)


> #### Display Lat-Lon and Time Bounds

# Visualize Data Cube Region

> #### Picking a smaller analysis region

In [8]:
## The code below renders a map that can be used to orient yourself with the region.
from utils.data_cube_utilities.dc_display_map import display_map
display_map(latitude = latitude_extents, longitude = longitude_extents)

In [9]:
######### Vietnam - Buan Tua Srah Lake ################## 
longitude_extents = (108.02, 108.15)
latitude_extents  = (12.18 , 12.30)

time_extents = ('2015-01-01', '2016-01-01')
print ( time_extents )

('2015-01-01', '2016-01-01')


In [10]:
from utils.data_cube_utilities.dc_display_map import display_map

display_map(latitude = latitude_extents, longitude = longitude_extents)

In [11]:
landsat_dataset = dc.load(latitude = latitude_extents,
                          longitude = longitude_extents,
                          platform = platform,
                          time = time_extents,
                          product = product,
                          measurements = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'pixel_qa']) 

In [28]:
landsat_dataset

<xarray.Dataset>
Dimensions:    (latitude: 446, longitude: 483, time: 19)
Coordinates:
  * time       (time) datetime64[ns] 2015-01-09T03:06:13 2015-01-25T03:06:16 ...
  * latitude   (latitude) float64 12.3 12.3 12.3 12.3 12.3 12.3 12.3 12.3 ...
  * longitude  (longitude) float64 108.0 108.0 108.0 108.0 108.0 108.0 108.0 ...
Data variables:
    red        (time, latitude, longitude) int16 912 1008 1125 989 951 1317 ...
    green      (time, latitude, longitude) int16 674 779 842 863 801 990 969 ...
    blue       (time, latitude, longitude) int16 493 473 574 554 473 714 714 ...
    nir        (time, latitude, longitude) int16 2500 2544 2587 2544 2630 ...
    swir1      (time, latitude, longitude) int16 2740 3002 3054 2871 2662 ...
    swir2      (time, latitude, longitude) int16 1678 1982 2037 1872 1677 ...
    pixel_qa   (time, latitude, longitude) int32 66 66 66 66 66 66 66 66 66 ...
Attributes:
    crs:      EPSG:4326

# Derive Several Products

>### Unpack pixel_qa

In [12]:
import xarray as xr  
import numpy as np

def ls8_unpack_qa( data_array , cover_type):  
    
    land_cover_endcoding = dict( fill         =[1] ,
                                 clear        =[322, 386, 834, 898, 1346],
                                 water        =[324, 388, 836, 900, 1348],
                                 shadow       =[328, 392, 840, 904, 1350],
                                 snow         =[336, 368, 400, 432, 848, 880, 812, 944, 1352],
                                 cloud        =[352, 368, 416, 432, 848, 880, 912, 944, 1352],
                                 low_conf_cl  =[322, 324, 328, 336, 352, 368, 834, 836, 840, 848, 864, 880],
                                 med_conf_cl  =[386, 388, 392, 400, 416, 432, 898, 900, 904, 928, 944],
                                 high_conf_cl =[480, 992],
                                 low_conf_cir =[322, 324, 328, 336, 352, 368, 386, 388, 392, 400, 416, 432, 480],
                                 high_conf_cir=[834, 836, 840, 848, 864, 880, 898, 900, 904, 912, 928, 944], 
                                 terrain_occ  =[1346,1348, 1350, 1352]
                               )
    
    boolean_mask = np.isin(data_array, land_cover_endcoding[cover_type]) 
    return xr.DataArray(boolean_mask.astype(int),
                        coords = data_array.coords,
                        dims = data_array.dims,
                        name = cover_type + "_mask",
                        attrs = data_array.attrs)   

In [13]:
clear_xarray  = ls8_unpack_qa(landsat_dataset.pixel_qa, "clear")  
water_xarray  = ls8_unpack_qa(landsat_dataset.pixel_qa, "water")

shadow_xarray = ls8_unpack_qa(landsat_dataset.pixel_qa, "shadow")  

In [14]:
clean_xarray = xr.ufuncs.logical_or(clear_xarray , water_xarray).astype(np.int8).rename("clean_mask")

clean_mask = np.logical_or(clear_xarray.values.astype(bool),
                           water_xarray.values.astype(bool)) 

> ### Water

In [15]:
from utils.data_cube_utilities.dc_water_classifier import wofs_classify

water_classification = wofs_classify(landsat_dataset,
                                     clean_mask = clean_mask, 
                                     mosaic = False) 

/home/localuser/Datacube/data_cube_notebooks/utils/data_cube_utilities/dc_water_classifier.py:76: RuntimeWarning: divide by zero encountered in true_divide
  return (a - b) / (a + b)
/home/localuser/Datacube/data_cube_notebooks/utils/data_cube_utilities/dc_water_classifier.py:76: RuntimeWarning: invalid value encountered in true_divide
  return (a - b) / (a + b)
/home/localuser/Datacube/data_cube_notebooks/utils/data_cube_utilities/dc_water_classifier.py:106: RuntimeWarning: invalid value encountered in less_equal
  r4 = ndi_43 <= 0.61
/home/localuser/Datacube/data_cube_notebooks/utils/data_cube_utilities/dc_water_classifier.py:113: RuntimeWarning: invalid value encountered in less_equal
  r6 = ndi_43 <= -0.01
/home/localuser/Datacube/data_cube_notebooks/utils/data_cube_utilities/dc_water_classifier.py:128: RuntimeWarning: invalid value encountered in less_equal
  r9 = ndi_43 <= 0.22
/home/localuser/Datacube/data_cube_notebooks/utils/data_cube_utilities/dc_water_classifier.py:152: Runt

In [16]:
wofs_xarray = water_classification.wofs

> ###  Normalized Indices  

In [17]:
def NDVI(dataset):
    return ((dataset.nir - dataset.red)/(dataset.nir + dataset.red)).rename("NDVI")

In [18]:
def NDWI(dataset):
    return ((dataset.green - dataset.nir)/(dataset.green + dataset.nir)).rename("NDWI")

In [19]:
def NDBI(dataset):
        return ((dataset.swir2 - dataset.nir)/(dataset.swir2 + dataset.nir)).rename("NDBI")

In [20]:
ndbi_xarray = NDBI(landsat_dataset)  # Urbanization - Reds
ndvi_xarray = NDVI(landsat_dataset)  # Dense Vegetation - Greens
ndwi_xarray = NDWI(landsat_dataset)  # High Concentrations of Water - Blues  

/home/localuser/Datacube/datacube_env/lib/python3.5/site-packages/xarray-0.9.6-py3.5.egg/xarray/core/variable.py:1164: RuntimeWarning: divide by zero encountered in true_divide
  if not reflexive
/home/localuser/Datacube/datacube_env/lib/python3.5/site-packages/xarray-0.9.6-py3.5.egg/xarray/core/variable.py:1164: RuntimeWarning: invalid value encountered in true_divide
  if not reflexive


>### TSM  

In [21]:
from utils.data_cube_utilities.dc_water_quality import tsm

tsm_xarray = tsm(landsat_dataset, clean_mask = wofs_xarray.values.astype(bool) ).tsm

/home/localuser/Datacube/datacube_env/lib/python3.5/site-packages/xarray-0.9.6-py3.5.egg/xarray/core/variable.py:1164: RuntimeWarning: invalid value encountered in power
  if not reflexive


> ### EVI  

In [22]:
def EVI(dataset, c1 = None, c2 = None, L = None):
        return ((dataset.nir - dataset.red)/((dataset.nir  + (c1 * dataset.red) - (c2 *dataset.blue) + L))).rename("EVI")

In [23]:
evi_xarray = EVI(landsat_dataset, c1 = 6, c2 = 7.5, L = 1 )

/home/localuser/Datacube/datacube_env/lib/python3.5/site-packages/xarray-0.9.6-py3.5.egg/xarray/core/variable.py:1164: RuntimeWarning: divide by zero encountered in true_divide
  if not reflexive
/home/localuser/Datacube/datacube_env/lib/python3.5/site-packages/xarray-0.9.6-py3.5.egg/xarray/core/variable.py:1164: RuntimeWarning: invalid value encountered in true_divide
  if not reflexive


# Combine Everything  

In [24]:
combined_dataset = xr.merge([landsat_dataset,
          clean_xarray,
          clear_xarray,
          water_xarray,
          shadow_xarray,
          evi_xarray,
          ndbi_xarray,
          ndvi_xarray,
          ndwi_xarray,
          wofs_xarray,
          tsm_xarray])

# Copy original crs to merged dataset 
combined_dataset = combined_dataset.assign_attrs(landsat_dataset.attrs)

combined_dataset

<xarray.Dataset>
Dimensions:      (latitude: 446, longitude: 483, time: 20)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01T03:07:41 ...
  * latitude     (latitude) float64 12.3 12.3 12.3 12.3 12.3 12.3 12.3 12.3 ...
  * longitude    (longitude) float64 108.0 108.0 108.0 108.0 108.0 108.0 ...
Data variables:
    red          (time, latitude, longitude) float32 782.0 789.0 890.0 787.0 ...
    green        (time, latitude, longitude) float32 753.0 773.0 859.0 792.0 ...
    blue         (time, latitude, longitude) float32 482.0 479.0 535.0 442.0 ...
    nir          (time, latitude, longitude) float32 2875.0 2947.0 2944.0 ...
    swir1        (time, latitude, longitude) float32 2318.0 2307.0 2308.0 ...
    swir2        (time, latitude, longitude) float32 1405.0 1398.0 1445.0 ...
    pixel_qa     (time, latitude, longitude) int32 322 322 322 322 322 322 ...
    clean_mask   (time, latitude, longitude) int8 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
    clear_mask   (time, latitude, longi

# Export Geotiff

----  
File formatting  

In [25]:
import time
def time_to_string(t):
    return time.strftime("%Y_%m_%d_%H_%M_%S", time.gmtime(t.astype(int)/1000000000))

----  
This function can be used to write a single time slice from an xarray to geotiff

In [26]:
from utils.data_cube_utilities import dc_utilities
def export_slice_to_geotiff(ds, path):
    dc_utilities.write_geotiff_from_xr(path,
                                        ds.astype(np.float32),
                                        list(combined_dataset.data_vars.keys()),
                                        crs="EPSG:4326")

----  
For each time slice in a dataset we call `export_slice_to_geotif`  

In [30]:
def export_xarray_to_geotiff(ds, path):
    for t in ds.time:
        time_slice_xarray = ds.sel(time = t)
        export_slice_to_geotiff(time_slice_xarray,
                                path + "_" + time_to_string(t) + ".tif")

----  
Start Export

In [31]:
export_xarray_to_geotiff(combined_dataset, "geotiffs/landsat8")

----  
Sanity check using `gdalinfo` to make sure that all of our bands exist    

In [34]:
!gdalinfo geotiffs/landsat8_2015_01_01_03_07_41.tif

Driver: GTiff/GeoTIFF
Files: geotiffs/landsat8_2015_01_01_03_07_41.tif
Size is 483, 446
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
Origin = (108.020032379927088,12.299867617463660)
Pixel Size = (0.000268936625432,-0.000268890337287)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  INTERLEAVE=PIXEL
Corner Coordinates:
Upper Left  ( 108.0200324,  12.2998676) (108d 1'12.12"E, 12d17'59.52"N)
Lower Left  ( 108.0200324,  12.1799425) (108d 1'12.12"E, 12d10'47.79"N)
Upper Right ( 108.1499288,  12.2998676) (108d 8'59.74"E, 12d17'59.52"N)
Lower Right ( 108.1499288,  12.1799425) (108d 8'59.74"E, 12d10'47.79"N)
Center      ( 108.0849806,  12.2399051) (108d 5' 5.93"E, 12d14'23.66"N)
Band 1 Block=483x1 Type=Float32, ColorInterp=Gray
  NoData Value=-9999
Ban

----  
Check to see what files exist in `geotiffs`

In [35]:
!ls -lah geotiffs/*.tif

-rw-rw-r-- 1 localuser localuser 14M Jan 25 20:49 geotiffs/landsat8_2015_01_01_03_07_41.tif
-rw-rw-r-- 1 localuser localuser 14M Jan 25 20:49 geotiffs/landsat8_2015_01_17_03_07_40.tif
-rw-rw-r-- 1 localuser localuser 14M Jan 25 20:49 geotiffs/landsat8_2015_02_02_03_07_36.tif
-rw-rw-r-- 1 localuser localuser 14M Jan 25 20:49 geotiffs/landsat8_2015_02_18_03_07_28.tif
-rw-rw-r-- 1 localuser localuser 14M Jan 25 20:49 geotiffs/landsat8_2015_03_06_03_07_22.tif
-rw-rw-r-- 1 localuser localuser 14M Jan 25 20:49 geotiffs/landsat8_2015_03_22_03_07_13.tif
-rw-rw-r-- 1 localuser localuser 14M Jan 25 20:49 geotiffs/landsat8_2015_04_07_03_07_02.tif
-rw-rw-r-- 1 localuser localuser 14M Jan 25 20:49 geotiffs/landsat8_2015_04_23_03_07_00.tif
-rw-rw-r-- 1 localuser localuser 14M Jan 25 20:49 geotiffs/landsat8_2015_05_09_03_06_44.tif
-rw-rw-r-- 1 localuser localuser 14M Jan 25 20:49 geotiffs/landsat8_2015_05_25_03_06_43.tif
-rw-rw-r-- 1 localuser localuser 14M Jan 25 20:49 geotiffs/landsat8_2015_06_10_0

----  
Zip all geotiffs  

In [36]:
!tar -cvzf geotiffs/landsat_8.tar.gz geotiffs/*.tif

geotiffs/landsat8_2015_01_01_03_07_41.tif
geotiffs/landsat8_2015_01_17_03_07_40.tif
geotiffs/landsat8_2015_02_02_03_07_36.tif
geotiffs/landsat8_2015_02_18_03_07_28.tif
geotiffs/landsat8_2015_03_06_03_07_22.tif
geotiffs/landsat8_2015_03_22_03_07_13.tif
geotiffs/landsat8_2015_04_07_03_07_02.tif
geotiffs/landsat8_2015_04_23_03_07_00.tif
geotiffs/landsat8_2015_05_09_03_06_44.tif
geotiffs/landsat8_2015_05_25_03_06_43.tif
geotiffs/landsat8_2015_06_10_03_06_54.tif
geotiffs/landsat8_2015_07_28_03_07_16.tif
geotiffs/landsat8_2015_08_13_03_07_21.tif
geotiffs/landsat8_2015_08_29_03_07_27.tif
geotiffs/landsat8_2015_09_30_03_07_40.tif
geotiffs/landsat8_2015_10_16_03_07_40.tif
geotiffs/landsat8_2015_11_01_03_07_46.tif
geotiffs/landsat8_2015_11_17_03_07_47.tif
geotiffs/landsat8_2015_12_03_03_07_48.tif
geotiffs/landsat8_2015_12_19_03_07_48.tif


----  
List files again to see the size of the gif created

In [37]:
!ls -lah geotiffs/

total 414M
drwxrwxr-x  3 localuser localuser 4.0K Jan 25 20:49 .
drwxrwxrwx 10 localuser localuser 4.0K Jan 25 20:49 ..
drwxr-xr-x  2 localuser localuser 4.0K Jan 25 20:04 .ipynb_checkpoints
-rw-rw-r--  1 localuser localuser  14M Jan 25 20:49 landsat8_2015_01_01_03_07_41.tif
-rw-rw-r--  1 localuser localuser  14M Jan 25 20:49 landsat8_2015_01_17_03_07_40.tif
-rw-rw-r--  1 localuser localuser  14M Jan 25 20:49 landsat8_2015_02_02_03_07_36.tif
-rw-rw-r--  1 localuser localuser  14M Jan 25 20:49 landsat8_2015_02_18_03_07_28.tif
-rw-rw-r--  1 localuser localuser  14M Jan 25 20:49 landsat8_2015_03_06_03_07_22.tif
-rw-rw-r--  1 localuser localuser  14M Jan 25 20:49 landsat8_2015_03_22_03_07_13.tif
-rw-rw-r--  1 localuser localuser  14M Jan 25 20:49 landsat8_2015_04_07_03_07_02.tif
-rw-rw-r--  1 localuser localuser  14M Jan 25 20:49 landsat8_2015_04_23_03_07_00.tif
-rw-rw-r--  1 localuser localuser  14M Jan 25 20:49 landsat8_2015_05_09_03_06_44.tif
-rw-rw-r--  1 localuser localuser  14M Jan 2